# Векторизаторы из библиотеки scikit-learn

В этом тюториале мы научимся:
- пользоваться векторизаторами из библиотеки _scikit-learn_
- превращать коллекцию текстов в матрицу термин-документ
- векторизовывать тексты в виде векторов взвешенных по TF-IDF
- ранжировать документы с помощью TF-IDF

Импортируем модули которые нам понадобятся впоследствии:

In [2]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import pairwise

Предположим, что на входе нам дана коллекция из нескольких текстовых документов (названия институтов):

In [3]:
texts = [
    "Московская государственная академия хореографии",
    "Московский государственный университет им. М.В. Ломоносова (Университет МГУ)",
    "Московский физико-технический институт (национальный исследовательский университет)",
    "Национальный исследовательский университет «МИЭТ»",
    "Национальный исследовательский университет ИТМО",
]
print(texts)

['Московская государственная академия хореографии', 'Московский государственный университет им. М.В. Ломоносова (Университет МГУ)', 'Московский физико-технический институт (национальный исследовательский университет)', 'Национальный исследовательский университет «МИЭТ»', 'Национальный исследовательский университет ИТМО']


## Матрица термин-документ

Превратим нашу коллекцию текстов в матрицу термин-документ с помощью класса _CountVectorizer_ из библиотеки _scikit_learn_.

Сначала нам потребуется создать объект класса:

In [4]:
vectorizer = CountVectorizer(min_df=1, binary=True)

Передаем в конструктор объекта параметры:
- _min_df_ -- порог на документную частоту, т.е. на число документов, в которых встречается данное слово. Векторизатор будет игнорировать редкие слова с частотой меньше, чем min_df. Мы, в данном случае, хотим оставить все слова.
- _binary_ -- бинаризовать TFы (частоты слов). По дефолту векторизатор хранит в каждой ячейке матрицы термин-документ TF, т.е. сколько раз данный термин встречается в данном документе. Параметр binary=True заставляет векторизатор использовать вместо TFов значения 0 или 1.

Полный список возможных параметров (их множество!) можно найти на страничке с документацией: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Теперь "обучим" наш векторизатор на текстах, и распечатаем получившуюся матрицу термин-документ:

In [5]:
X = vectorizer.fit_transform(texts)
print(X)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 25 stored elements and shape (5, 17)>
  Coords	Values
  (0, 10)	1
  (0, 1)	1
  (0, 0)	1
  (0, 16)	1
  (1, 11)	1
  (1, 2)	1
  (1, 14)	1
  (1, 3)	1
  (1, 7)	1
  (1, 8)	1
  (2, 11)	1
  (2, 14)	1
  (2, 15)	1
  (2, 13)	1
  (2, 4)	1
  (2, 12)	1
  (2, 5)	1
  (3, 14)	1
  (3, 12)	1
  (3, 5)	1
  (3, 9)	1
  (4, 14)	1
  (4, 12)	1
  (4, 5)	1
  (4, 6)	1


Обратим тут внимание на два момента.

Во-первых, мы вызываем функцию _fit_transform()_, т.е. сразу и обучаем векторизатор, и векторизуем наши тексты. Эти два этапа можно разделить, т.е. сначала обучать векторизатор вызовом _fit()_, а потом применять его для векторизации каких-то других текстов с помощью функции _transform_(). Дальше мы увидим как все это работает на примерах.

Во-вторых, на выходе у нас не обычный numpy array, а что-то странное.<br>
Посмотрим на тип объекта X:

In [6]:
print(type(X))

<class 'scipy.sparse._csr.csr_matrix'>


Видим, что тут используются классы для представления разреженных (sparse) матриц из библиотеки scipy.

Эти классы описаны в документации: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

Мотивация тут очень простая -- мы векторизуем наши тексты так, что:
- каждый текст представляется в виде вектора длиной размера словаря, т.е. для большой коллекции текстов это могут быть сотни тысяч или даже миллионы слов (терминов)
- в каждом конкретном векторе у нас выставлены в 1 только те элементы, которые соответствуют словам, которые были в данном тексте, а их, как правило, гораздо меньше чем полное количество известных нам слов => вектора получаются очень разреженные.

Посмотрим еще раз на размеры нашей матрицы:

In [7]:
print(X.shape)

(5, 17)


Т.е. у нас всего 5 документов и 17 уникальных слов в словаре.

А сколько у нас ненулевых элементов?

In [8]:
print(X.nnz)

25


У нас всего 265 ненулевых элемента, т.е. матрица заполнена всего на 25 / (5 * 17) = 29.4%.

А если бы наша коллекция была больше, то эта заполненность была бы еще во много-много раз ниже.

Посмотрим на нашу матрицу в более удобном виде:

In [9]:
print(X.toarray())

[[1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1]
 [0 0 1 1 0 0 0 1 1 0 0 1 0 0 1 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 1 1 1 1 1 0]
 [0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0]
 [0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0]]


Теперь мы видим ее в типичном представлении матрицы термин-документ, где:
- каждая строка соответствует документу
- каждый столбец соответствует слову из словаря

Как понять каким конкретно словам соответсвуют столбцы?

Для этого можно воспользоваться свойством векторизатора vocabulary_:

In [10]:
print(vectorizer.vocabulary_)

{'московская': 10, 'государственная': 1, 'академия': 0, 'хореографии': 16, 'московский': 11, 'государственный': 2, 'университет': 14, 'им': 3, 'ломоносова': 7, 'мгу': 8, 'физико': 15, 'технический': 13, 'институт': 4, 'национальный': 12, 'исследовательский': 5, 'миэт': 9, 'итмо': 6}


Мы видим, что словарь представляет из себя словарь, отображающий слово в позицию, т.е. например:
- слово "академия" соответсвует 0-му столбцу
- слово "государственная" 1-му и т.д.

Также, словарь можно распечатать в уже упорядоченном виде с помощью метода _get_feature_names_out()_:

In [11]:
print(vectorizer.get_feature_names_out())

['академия' 'государственная' 'государственный' 'им' 'институт'
 'исследовательский' 'итмо' 'ломоносова' 'мгу' 'миэт' 'московская'
 'московский' 'национальный' 'технический' 'университет' 'физико'
 'хореографии']


Обратим внимание, что в этом списке слов есть далеко не все слова исходных текстов, напр. куда-то пропали однобуквенные слова из названия института "университет им. М.В. Ломоносова".

Дело в том, что наш векторизатор "под капотом" предобрабатывает текст, в т.ч.:
- понижает регистр
- токенизирует текст с помощью встроенного токенизатора

Все эти этапы, при желании, можно кастомизировать.

Предположим, что мы не хотим терять однословные слова. В этом случае будет достаточно кастомизировать токенизатор, передав ему наш собственный _token_pattern_:

In [12]:
# Rewrite default token pattern '(?u)\\b\\w\\w+\\b'
vectorizer = CountVectorizer(min_df=1, token_pattern='(?u)\\b\\w+\\b', binary=True)
X = vectorizer.fit_transform(texts)
print(X.shape)
print(vectorizer.get_feature_names_out())

(5, 19)
['академия' 'в' 'государственная' 'государственный' 'им' 'институт'
 'исследовательский' 'итмо' 'ломоносова' 'м' 'мгу' 'миэт' 'московская'
 'московский' 'национальный' 'технический' 'университет' 'физико'
 'хореографии']


Теперь мы видим, что размер наших векторов стал больше, и в словаре остались однобуквенные слова.

## Векторы TF-IDF

Теперь мы можем воспользоваться другим из доступных в библиотеке _scikit-learn_ векторизаторов -- _TfidfVectorizer_, с помощью которого мы сможем представить нашу коллекцию текстов в виде векторов TF-IDF.

_TfidfVectorizer_ очень похож на _CountVectorizer_, в т.ч. аналогичным образом позволяет кастомизировать предобработку текстов.

Все детали можно найти в документации: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

Попробуем создать объект нашего векторизатора:

In [13]:
vectorizer = TfidfVectorizer(min_df=1, norm=None, smooth_idf=True)

Параметр norm=None говорит о том, что мы не хотим нормализовывать TF-IDF-векторы (по дефолту там используется L2-норма).

Векторизуем нашу коллекцию:

In [14]:
vectorizer.fit(texts)
X = vectorizer.transform(texts)
print(X.shape)
print(vectorizer.get_feature_names_out())

(5, 17)
['академия' 'государственная' 'государственный' 'им' 'институт'
 'исследовательский' 'итмо' 'ломоносова' 'мгу' 'миэт' 'московская'
 'московский' 'национальный' 'технический' 'университет' 'физико'
 'хореографии']


Мы видим, что на выходе получилась матрица точно такого же размера, что и в прошлый раз.

Однако ее элементы теперь выглядят по другому:

In [15]:
print(X.toarray())

[[2.09861229 2.09861229 0.         0.         0.         0.
  0.         0.         0.         0.         2.09861229 0.
  0.         0.         0.         0.         2.09861229]
 [0.         0.         2.09861229 2.09861229 0.         0.
  0.         2.09861229 2.09861229 0.         0.         1.69314718
  0.         0.         2.36464311 0.         0.        ]
 [0.         0.         0.         0.         2.09861229 1.40546511
  0.         0.         0.         0.         0.         1.69314718
  1.40546511 2.09861229 1.18232156 2.09861229 0.        ]
 [0.         0.         0.         0.         0.         1.40546511
  0.         0.         0.         2.09861229 0.         0.
  1.40546511 0.         1.18232156 0.         0.        ]
 [0.         0.         0.         0.         0.         1.40546511
  2.09861229 0.         0.         0.         0.         0.
  1.40546511 0.         1.18232156 0.         0.        ]]


Теперь в каждой ячейке матрицы лежат не значения 0 (слова нет в тексте) или 1 (слово есть в тексте), а значения TF-IDF.

А, с учетом того, что почти все TFы у нас равны 1, т.к. слова в текстах не повторяются (кроме слова УНИВЕРСИТЕТ во 2м документе), почти все TF-IDFы равны просто IDFам.

Заметим, что с помощью _TfidfVectorizer_ мы получили те же самые значения IDFов, что и в предыдущем упражнении, когда считали их самостоятельно!

## Ранжирование с помощью TF-IDF

Теперь перейдем к самому интересному: попробуем ранжировать наши документы, используя в качестве ранков косинусное расстояние между TF-IDF-векторами запросов и документов.

Допустим, у нас есть текстовый запрос query.

Напишем функцию _search(query)_, которая:
- векторизует наш запрос с помощью обученного ранее векторизатора
- считаем попарную близость между вектором запроса и векторами документов
- выводит на экран запрос и ранжированный список документов

In [16]:
def search(query):
        # Vectorize query
        X_query = vectorizer.transform([query])
        
        # Query-docs similarity
        S = pairwise.cosine_similarity(X_query, X)
        
        # Rank docs
        scores = S[0]
        indexes = np.argsort(scores)[::-1]
        ranked_docs = np.array(texts)[indexes]
        ranked_doc_scores = scores[indexes]

        # Output query and list of ranked docs
        print(f"query = '{query}'")
        for i, doc in enumerate(ranked_docs):
            score = ranked_doc_scores[i]
            print(f"[{i}]: doc = '{doc}' score = {score:.3f}")

Применим нашу функцию к запросу "университет":

In [17]:
search("университет")

query = 'университет'
[0]: doc = 'Московский государственный университет им. М.В. Ломоносова (Университет МГУ)' score = 0.463
[1]: doc = 'Национальный исследовательский университет ИТМО' score = 0.379
[2]: doc = 'Национальный исследовательский университет «МИЭТ»' score = 0.379
[3]: doc = 'Московский физико-технический институт (национальный исследовательский университет)' score = 0.255
[4]: doc = 'Московская государственная академия хореографии' score = 0.000


Видим, что на 1м месте у нас МГУ, как и ожидалось.

Попробуем другой запрос:

In [18]:
search("московский")

query = 'московский'
[0]: doc = 'Московский физико-технический институт (национальный исследовательский университет)' score = 0.366
[1]: doc = 'Московский государственный университет им. М.В. Ломоносова (Университет МГУ)' score = 0.332
[2]: doc = 'Национальный исследовательский университет ИТМО' score = 0.000
[3]: doc = 'Национальный исследовательский университет «МИЭТ»' score = 0.000
[4]: doc = 'Московская государственная академия хореографии' score = 0.000


Обратим внимание, что на 1м месте у нас МФТИ -- это связано с тем, что полное название МФТИ короче, чем полное название МГУ, а в формуле косинусной близости у нас используются длины наших векторов.

Таким образом, формула TF-IDF позволяет даже для однословных запросов отличать потенциально более релевантные документы от менее релевантных!

Попробуем теперь многословный запрос:

In [19]:
search("московский университет")

query = 'московский университет'
[0]: doc = 'Московский государственный университет им. М.В. Ломоносова (Университет МГУ)' score = 0.537
[1]: doc = 'Московский физико-технический институт (национальный исследовательский университет)' score = 0.446
[2]: doc = 'Национальный исследовательский университет ИТМО' score = 0.217
[3]: doc = 'Национальный исследовательский университет «МИЭТ»' score = 0.217
[4]: doc = 'Московская государственная академия хореографии' score = 0.000


Таким образом, мы научились ранжировать документы, используя модель векторного пространства и векторы TF-IDF!